In [157]:
import pandas as pd
import requests as r
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [158]:
api_str = 'https://api.llama.fi/protocol/'

protocols = [
    # name, incentive start date
         ['velodrome','2022-07-13']
        ,['pooltogether','2022-07-14']
        ,['lyra','2022-08-02']
        ,['rubicon','2022-07-15']
#         ,['perpetual-protocol','2022-07-14']
#         ,['thales','2022-07-14'] #TVL not relevant
        ,['aave-v3','2022-08-04']
        ]
# print(protocols[0])
prod = []
for prot in protocols:
    ad = pd.json_normalize( r.get(api_str + prot[0]).json()['chainTvls']['Optimism']['tokens'] )
    ad['date'] = pd.to_datetime(ad['date'], unit ='s') #convert to days
    ad = pd.melt(ad,id_vars = ['date'])
    ad = ad.rename(columns={'variable':'token'})
    ad['token'] = ad['token'].str.replace('tokens.','')
    ad['protocol'] = prot[0]
    ad['start_date'] = pd.to_datetime(prot[1])
    ad['date'] = ad['date'] - timedelta(days=1) #change to eod vs sod
    prod.append(ad)

df_df = pd.concat(prod)
df_df

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_25533/1926726922.py:20: FutureWarning:

The default value of regex will change from True to False in a future version.

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_25533/1926726922.py:20: FutureWarning:

The default value of regex will change from True to False in a future version.

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_25533/1926726922.py:20: FutureWarning:

The default value of regex will change from True to False in a future version.

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_25533/1926726922.py:20: FutureWarning:

The default value of regex will change from True to False in a future version.

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_25533/1926726922.py:20: FutureWarning:

The default value of regex will change from True to False in a future version.



,date,token,value,protocol,start_date
0,2022-06-01 00:00:00,SETH,0.02061,velodrome,2022-07-13
1,2022-06-02 00:00:00,SETH,0.02061,velodrome,2022-07-13
2,2022-06-03 00:00:00,SETH,0.02609,velodrome,2022-07-13
3,2022-06-04 00:00:00,SETH,0.03786,velodrome,2022-07-13
4,2022-06-05 00:00:00,SETH,0.03142,velodrome,2022-07-13
...,...,...,...,...,...
1163,2022-08-01 00:00:00,AAVE,354.22096,aave-v3,2022-08-04
1164,2022-08-02 00:00:00,AAVE,354.23596,aave-v3,2022-08-04
1165,2022-08-03 00:00:00,AAVE,354.23596,aave-v3,2022-08-04
1166,2022-08-04 00:00:00,AAVE,93710.56889,aave-v3,2022-08-04


In [159]:
# DISTINCT TOKENS

token_list = df_df[['token']].drop_duplicates()
token_list

,token
0,SETH
66,FXS
132,THALES
198,FRAX
264,USDT
330,WETH
396,ALUSD
462,DAI
528,SUSD
594,PERP


In [160]:
#defillama api feedback - only token symbols come through, makes it hard to map w/o doing it manually
coingecko_token_map = [
    ['LINK','chainlink']
    ,['USDT','tether']
    ,['USDC','usd-coin']
    ,['WETH','weth']
    ,['SUSD','nusd']
    ,['DAI','dai']
    ,['AAVE','aave']
    ,['WBTC','wrapped-bitcoin']
    ,['SNX','havven']
    ,['OP','optimism']
    ,['SETH','seth']
    ,['FXS','frax-share']
    ,['THALES','thales']
    ,['FRAX','frax']
    ,['ALUSD','alchemix-usd']
    ,['PERP','perpetual-protocol']
    ,['LUSD','liquity-usd']
    ,['LYRA','lyra-finance']
    ,['HND','hundred-finance']
    ,['BITANT','bitant']
    ,['UNI','uniswap']
    ,['SLINK','slink']
    ,['VELO','velodrome-finance']
    ,['DOLA','dola-usd']
    ,['CRV','curve-dao-token']
    ,['SBTC','sbtc']
    ,['KROM','kromatika']
    ,['DF','dforce-token']
    ,['STG','stargate-finance']
    ,['AELIN','aelin']
    ,['RAI','rai']
    ,['RETH','rocket-pool-eth']
]

cg_pds = []
for t in coingecko_token_map:
    cg_api = 'https://api.coingecko.com/api/v3/coins/'\
            + t[1] + '/market_chart?vs_currency=usd&days=max&interval=daily'
    pr = pd.DataFrame( r.get(cg_api).json()['prices'] )
    pr['token'] = t[0]
    pr['cg_slug'] = t[1]
    pr = pr.rename(columns={0:'date',1:'price_usd'})
    pr['date'] = pd.to_datetime(pr['date'], unit ='ms') #convert to days
    cg_pds.append(pr)

cg_df = pd.concat(cg_pds)
# cg_df

In [161]:
data_df = df_df.merge(cg_df, on=['date','token'],how='inner')
data_df['net_token_flow'] = data_df.groupby(['token','protocol','start_date'])['value'].apply(lambda x: x- x.shift(1))
data_df['net_dollar_flow'] = data_df['net_token_flow'] * data_df['price_usd']

data_df

,date,token,value,protocol,start_date,price_usd,cg_slug,net_token_flow,net_dollar_flow
0,2022-06-01,SETH,0.02061,velodrome,2022-07-13,1946.863023,seth,NaN,NaN
1,2022-06-01,SETH,1558.13304,lyra,2022-08-02,1946.863023,seth,NaN,NaN
2,2022-06-02,SETH,0.02061,velodrome,2022-07-13,1828.527016,seth,0.00000,0.000000e+00
3,2022-06-02,SETH,1715.32274,lyra,2022-08-02,1828.527016,seth,157.18970,2.874256e+05
4,2022-06-03,SETH,0.02609,velodrome,2022-07-13,1841.067526,seth,0.00548,1.008905e+01
...,...,...,...,...,...,...,...,...,...
5775,2022-07-31,AAVE,354.20657,aave-v3,2022-08-04,95.032810,aave,0.01000,9.503281e-01
5776,2022-08-01,AAVE,354.22096,aave-v3,2022-08-04,98.008488,aave,0.01439,1.410342e+00
5777,2022-08-02,AAVE,354.23596,aave-v3,2022-08-04,96.975152,aave,0.01500,1.454627e+00
5778,2022-08-03,AAVE,354.23596,aave-v3,2022-08-04,94.255863,aave,0.00000,0.000000e+00


In [162]:
netdf_df = data_df[data_df['date']>= data_df['start_date']][['date','protocol','net_dollar_flow']]
netdf_df = netdf_df.groupby(['date','protocol']).sum(['net_dollar_flow'])
netdf_df['cumul_net_dollar_flow'] = netdf_df.groupby(['protocol']).cumsum()
netdf_df.reset_index(inplace=True)

netdf_df

,date,protocol,net_dollar_flow,cumul_net_dollar_flow
0,2022-07-13,velodrome,1.289995e+06,1.289995e+06
1,2022-07-14,pooltogether,-1.568344e+04,-1.568344e+04
2,2022-07-14,velodrome,5.067062e+06,6.357057e+06
3,2022-07-15,pooltogether,1.196553e+05,1.039718e+05
4,2022-07-15,rubicon,7.484735e+04,7.484735e+04
...,...,...,...,...
65,2022-08-04,aave-v3,2.399380e+08,2.399380e+08
66,2022-08-04,lyra,1.854110e+06,8.049828e+06
67,2022-08-04,pooltogether,3.196015e+06,2.581713e+07
68,2022-08-04,rubicon,1.424911e+05,1.858258e+06


In [166]:
fig = px.bar(netdf_df, x="date", y="net_dollar_flow", color="protocol", \
             title="Daily Net Dollar Flow since Program Announcement",\
            labels={
                     "date": "Day",
                     "net_dollar_flow": "Net Dollar Flow (N$F)"
                 }
            )
cumul_fig.update_layout(
    legend_title="App Name"
)
fig.update_layout(yaxis_tickprefix = '$')
fig.show()

# cumul_fig = px.area(netdf_df, x="date", y="cumul_net_dollar_flow", color="protocol", \
#              title="Cumulative Dollar Flow since Program Announcement",\
#                    labels={
#                      "date": "Day",
#                      "cumul_net_dollar_flow": "Cumulative Net Dollar Flow (N$F)"
#                  }
#             ,areamode='group')
# cumul_fig.update_layout(yaxis_tickprefix = '$')
# cumul_fig.show()


cumul_fig = go.Figure()
proto_names = netdf_df['protocol'].drop_duplicates()
print(proto_names)
for p in proto_names:
    cumul_fig.add_trace(go.Scatter(x=netdf_df[netdf_df['protocol'] == p]['date'] \
                                   , y=netdf_df[netdf_df['protocol'] == p]['cumul_net_dollar_flow'] \
                                    ,name = p\
                                  ,fill='tozeroy')) # fill down to xaxis

cumul_fig.update_layout(yaxis_tickprefix = '$')
cumul_fig.update_layout(
    title="Cumulative Dollar Flow since Program Announcement",
    xaxis_title="Day",
    yaxis_title="Cumulative Net Dollar Flow (N$F)",
    legend_title="App Name",
#     color_discrete_map=px.colors.qualitative.G10
)
cumul_fig.show()

0        velodrome
1     pooltogether
4          rubicon
57            lyra
65         aave-v3
Name: protocol, dtype: object
